It does not work without internet, but I tried at least.

In [ ]:
#REQUIRES INTERNET
!pip install transformers accelerate bitsandbytes

In [ ]:
#REQUIRES INTERNET
!pip install --upgrade huggingface_hub

In [ ]:
import numpy as np
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# REQUIRES INTERNET
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

access_token_read = UserSecretsClient().get_secret("HUGGINGFACE_TOKEN")
login(token = access_token_read)

In [ ]:
import pandas as pd

trainDF = pd.read_csv("/kaggle/input/llm-prompt-recovery/train.csv")
testDF = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
sampleSubmission = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")

In [ ]:
trainDF

In [ ]:
trainDF.loc[0, 'original_text'], trainDF.loc[0, 'rewrite_prompt'], trainDF.loc[0, 'rewritten_text']

In [ ]:
testDF

In [ ]:
testDF.loc[0, 'original_text'], testDF.loc[0, 'rewritten_text']

In [ ]:
sampleSubmission

In [ ]:
def createSystemPrompt(orgText, modText):
    systemPrompt = (
        "Start with the initial text and craft a guiding prompt to transform it into its modified version.\n" 
        "Your objective is to preserve the essence while elevating the language and structure for greater clarity and impact.\n"
        "The guiding prompt must embody precision, coherence, and quality.\n"

        "Example:\n"
        "- Initial Text: " + trainDF.loc[0, 'original_text'] +
        ",\n- Modified Text: " + trainDF.loc[0, 'rewritten_text'] +
        ",\n- Suitable Guiding Prompt: " + trainDF.loc[0, 'rewrite_prompt'] +

        "\nYour task is to generate the most fitting guiding prompt based on the given original and modified texts."
        "- Original Text:\n" +
        "{orgText} \n" +
        "- Modified Text: \n" +
        "{modText}"
    ).format(orgText=orgText, modText=modText)
    return systemPrompt

In [ ]:
orgText = testDF.loc[0, 'original_text']
modText = testDF.loc[0, 'rewritten_text']

sysPrompt = createSystemPrompt(orgText, modText)
print(sysPrompt)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

modelName = "/kaggle/input/gemma/transformers/7b-it/2"

bnbConfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(modelName)

model = AutoModelForCausalLM.from_pretrained(
    modelName,
    device_map = "auto",
    quantization_config=bnbConfig
)

In [ ]:
def predictRewritePrompt(orgText, rewText, model, tokenizer):
    prompt = f"Original Text: {orgText} \n Modified Text: {rewText} \n Rewrite Prompt: "
    
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1)
    
    predicted_prompt = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return predicted_prompt

In [ ]:
predPrompts = []
for index, row in sampleSubmission.iterrows():
    predPrompt = predictRewritePrompt(row['original_text'], row['rewritten_text'], model, tokenizer)
    predPrompts.append(predPrompt)
    
sampleSubmission['rewrite_prompt'] = predPrompts

In [ ]:
sampleSubmission.loc[0, 'rewrite_prompt']

In [ ]:
sampleSubmission.to_csv('submission.csv', index=False)